# Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy

# Collaborative filtering approach
This method finds other users similar to you and recommends their choices. For example, if you and a friend both love The Shawshank Redemption, and your friend also loves Forrest Gump, the system will recommend Forrest Gump to you, thinking you might share your friend's taste.

# Load data

In [2]:
data_movies_metadata = pd.read_csv('movies_metadata.csv')
data_ratings = pd.read_csv('ratings_small.csv')

C:\Users\thdam\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Merge datasets

In [3]:
data_ratings['movieId'] = data_ratings['movieId'].astype(str)
data_movies_metadata['id'] = data_movies_metadata['id'].astype(str)
merged_data = pd.merge(data_ratings, data_movies_metadata, left_on='movieId', right_on='id', how='inner')
merged_data.head()

,userId,movieId,rating,timestamp,adult,belongs_to_collection,budget,genres,homepage,id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,1371,2.5,1260759135,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",NaN,1371,...,1982-05-28,270000000.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894.0
1,4,1371,4.0,949810302,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",NaN,1371,...,1982-05-28,270000000.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894.0
2,7,1371,3.0,851869160,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",NaN,1371,...,1982-05-28,270000000.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894.0
3,19,1371,4.0,855193404,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",NaN,1371,...,1982-05-28,270000000.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894.0
4,21,1371,3.0,853852263,False,"{'id': 1575, 'name': 'Rocky Collection', 'post...",17000000,"[{'id': 18, 'name': 'Drama'}]",NaN,1371,...,1982-05-28,270000000.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The greatest challenge.,Rocky III,False,6.6,894.0


# Preprocessing data

In [4]:
merged_data.isnull().sum()

userId                       0
movieId                      0
rating                       0
timestamp                    0
adult                        0
belongs_to_collection    34201
budget                       0
genres                       0
homepage                 34035
id                           0
imdb_id                      0
original_language            0
original_title               0
overview                   137
popularity                   0
poster_path                 36
production_companies         0
production_countries         0
release_date                29
revenue                      0
runtime                      4
spoken_languages             0
status                       1
tagline                  13478
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

In [5]:
merged_data.describe(include='all')

,userId,movieId,rating,timestamp,adult,belongs_to_collection,budget,genres,homepage,id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
count,44994.000000,44994,44994.000000,4.499400e+04,44994,10793,44994,44994,10959,44994,...,44965,4.499400e+04,44990.000000,44994,44993,31516,44994,44994,44994.000000,44994.000000
unique,NaN,2830,NaN,NaN,2,345,313,790,572,2830,...,2444,NaN,NaN,404,4,1786,2794,2,NaN,NaN
top,NaN,296,NaN,NaN,False,"{'id': 528, 'name': 'The Terminator Collection...",0,"[{'id': 18, 'name': 'Drama'}]",http://www.sonypictures.com/homevideo/meninbla...,296,...,2003-07-02,NaN,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Machines Will Rise.,Terminator 3: Rise of the Machines,False,NaN,NaN
freq,NaN,324,NaN,NaN,44993,386,19523,4953,224,324,...,324,NaN,NaN,22459,44939,324,324,44990,NaN,NaN
mean,345.963395,NaN,3.560986,1.089527e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.613321e+07,108.985441,NaN,NaN,NaN,NaN,NaN,6.634178,669.509490
std,194.871031,NaN,1.053169,1.878648e+08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.759024e+08,24.917268,NaN,NaN,NaN,NaN,NaN,1.059538,1219.402596
min,1.000000,NaN,0.500000,7.896520e+08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,177.000000,NaN,3.000000,9.472927e+08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,95.000000,NaN,NaN,NaN,NaN,NaN,6.100000,31.000000
50%,359.000000,NaN,4.000000,1.053172e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.201126e+06,106.000000,NaN,NaN,NaN,NaN,NaN,6.800000,176.000000
75%,518.000000,NaN,4.000000,1.227918e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.418968e+07,121.000000,NaN,NaN,NaN,NaN,NaN,7.300000,672.000000


In [6]:
merged_data=merged_data.sort_values(by='userId')

# Split data

In [7]:
reader = Reader(rating_scale=(1, 5))

In [8]:
data = Dataset.load_from_df(merged_data[['userId', 'movieId', 'rating']], reader)

In [9]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Singular Value Decomposition (SVD)
The SVD algorithm is trained on the user-movie ratings. It learns latent features for both users and movies, which it uses to predict unseen ratings

# Train

In [10]:
# Use the SVD algorithm for matrix factorization
model = SVD()

# Train the model on the training data
model.fit(trainset)

# Evaluate the model on the test data
predictions = model.test(testset)
svd_rmse =accuracy.rmse(predictions)

RMSE: 0.8899


# Make prediction

In [11]:
# Get all unique users and movies
unique_users = merged_data['userId'].unique()
unique_movies = merged_data['movieId'].unique()

# Dictionary to store top N recommendations for each user
top_n_recommendations = {}

# Predict ratings for each user-movie pair
for user in unique_users:
    user_recommendations = []
    for movie in unique_movies:
        # Predict rating for the (user, movie) pair
        pred = model.predict(user, movie, verbose=False)
        user_recommendations.append((movie, pred.est))
    
    # Sort movies by predicted rating and get top N (e.g., top 5)
    top_n = sorted(user_recommendations, key=lambda x: x[1], reverse=True)[:5]
    
    # Store top N recommendations for the user
    top_n_recommendations[user] = top_n

# Output top N recommendations for each user
for user, recommendations in top_n_recommendations.items():
    print(f"Top 5 recommendations for User {user}:")
    for movie, predicted_rating in recommendations:
        print(f"Movie {movie}: Predicted Rating {predicted_rating:.2f}")


Top 5 recommendations for User 1:
Movie 926: Predicted Rating 4.23
Movie 6016: Predicted Rating 4.21
Movie 318: Predicted Rating 4.17
Movie 913: Predicted Rating 4.07
Movie 1247: Predicted Rating 4.05
Top 5 recommendations for User 2:
Movie 926: Predicted Rating 4.61
Movie 913: Predicted Rating 4.48
Movie 923: Predicted Rating 4.47
Movie 6016: Predicted Rating 4.44
Movie 1250: Predicted Rating 4.35
Top 5 recommendations for User 3:
Movie 318: Predicted Rating 4.27
Movie 913: Predicted Rating 4.22
Movie 926: Predicted Rating 4.20
Movie 912: Predicted Rating 4.17
Movie 905: Predicted Rating 4.17
Top 5 recommendations for User 4:
Movie 318: Predicted Rating 5.00
Movie 2064: Predicted Rating 5.00
Movie 858: Predicted Rating 5.00
Movie 913: Predicted Rating 5.00
Movie 1247: Predicted Rating 5.00
Top 5 recommendations for User 5:
Movie 926: Predicted Rating 4.77
Movie 922: Predicted Rating 4.76
Movie 318: Predicted Rating 4.68
Movie 913: Predicted Rating 4.65
Movie 1250: Predicted Rating 4.6

Movie 913: Predicted Rating 4.81
Movie 4993: Predicted Rating 4.79
Movie 527: Predicted Rating 4.77
Top 5 recommendations for User 572:
Movie 858: Predicted Rating 4.73
Movie 898: Predicted Rating 4.72
Movie 58559: Predicted Rating 4.72
Movie 318: Predicted Rating 4.68
Movie 913: Predicted Rating 4.65
Top 5 recommendations for User 573:
Movie 527: Predicted Rating 4.48
Movie 296: Predicted Rating 4.44
Movie 318: Predicted Rating 4.42
Movie 926: Predicted Rating 4.42
Movie 608: Predicted Rating 4.42
Top 5 recommendations for User 574:
Movie 858: Predicted Rating 4.66
Movie 318: Predicted Rating 4.56
Movie 913: Predicted Rating 4.48
Movie 926: Predicted Rating 4.46
Movie 1089: Predicted Rating 4.42
Top 5 recommendations for User 575:
Movie 913: Predicted Rating 4.55
Movie 593: Predicted Rating 4.52
Movie 1254: Predicted Rating 4.46
Movie 994: Predicted Rating 4.44
Movie 858: Predicted Rating 4.38
Top 5 recommendations for User 576:
Movie 318: Predicted Rating 4.61
Movie 4226: Predicted R

# K-Nearest Neighbors

In [12]:
from surprise import KNNBasic

In [13]:
knn_model = KNNBasic()
knn_model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
knn_predictions = knn_model.test(testset)
knn_rmse = accuracy.rmse(knn_predictions)
knn_rmse 

RMSE: 0.9505


0.9504570556350629

In [15]:
# Get all unique users and movies
unique_users = merged_data['userId'].unique()
unique_movies = merged_data['movieId'].unique()

# Dictionary to store top N recommendations for each user
top_n_recommendations = {}

# Predict ratings for each user-movie pair
for user in unique_users:
    user_recommendations = []
    for movie in unique_movies:
        # Predict rating for the (user, movie) pair
        pred = knn_model.predict(user, movie, verbose=False)
        user_recommendations.append((movie, pred.est))
    
    # Sort movies by predicted rating and get top N (e.g., top 5)
    top_n = sorted(user_recommendations, key=lambda x: x[1], reverse=True)[:5]
    
    # Store top N recommendations for the user
    top_n_recommendations[user] = top_n

# Output top N recommendations for each user
for user, recommendations in top_n_recommendations.items():
    print(f"Top 5 recommendations for User {user}:")
    for movie, predicted_rating in recommendations:
        print(f"Movie {movie}: Predicted Rating {predicted_rating:.2f}")

Top 5 recommendations for User 1:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 26131: Predicted Rating 5.00
Movie 6643: Predicted Rating 5.00
Top 5 recommendations for User 2:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789: Predicted Rating 5.00
Movie 93855: Predicted Rating 5.00
Top 5 recommendations for User 3:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789: Predicted Rating 5.00
Top 5 recommendations for User 4:
Movie 2086: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789: Predicted Rating 5.00
Movie 93855: Predicted Rating 5.00
Movie 50641: Predicted Rating 5.00
Top 5 recommendations for User 5:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789

Movie 4789: Predicted Rating 5.00
Top 5 recommendations for User 562:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789: Predicted Rating 5.00
Top 5 recommendations for User 563:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 4271: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Top 5 recommendations for User 564:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 4789: Predicted Rating 5.00
Top 5 recommendations for User 565:
Movie 2086: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movie 5062: Predicted Rating 5.00
Movie 73881: Predicted Rating 5.00
Top 5 recommendations for User 566:
Movie 318: Predicted Rating 5.00
Movie 5071: Predicted Rating 5.00
Movie 1859: Predicted Rating 5.00
Movi

# Model ranking

In [16]:
model_performance = {
    'SVD': svd_rmse,
    'k-NN': knn_rmse
}

# Rank models based on RMSE
ranked_models = sorted(model_performance.items(), key=lambda x: x[1])
print("Ranked models:", ranked_models)

Ranked models: [('SVD', 0.889905145840725), ('k-NN', 0.9504570556350629)]


# Hyperparameter tuning

In [17]:
from surprise.model_selection import GridSearchCV

In [18]:
# Hyperparameter tuning for SVD
param_grid_svd = {
    'n_factors': [10, 20, 30],
    'reg_all': [0.02, 0.05, 0.1],
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse'], cv=3)
grid_search_svd.fit(data)

# Best parameters for SVD
print("Best SVD parameters:", grid_search_svd.best_params)

Best SVD parameters: {'rmse': {'n_factors': 10, 'reg_all': 0.05}}


In [19]:
# Hyperparameter tuning for k-NN
param_grid_knn = {
    'k': [5, 10, 15],
    'sim_options': {'name': ['msd', 'cosine'], 'user_based': [True]}
}

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse'], cv=3)
grid_search_knn.fit(data)

# Best parameters for k-NN
print("Best k-NN parameters:", grid_search_knn.best_params)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matr

Final Model Training with Best Parameters

In [20]:
best_svd = SVD(n_factors=grid_search_svd.best_params['rmse']['n_factors'],
               reg_all=grid_search_svd.best_params['rmse']['reg_all'])
best_svd.fit(trainset)

# Train final k-NN model with best parameters
best_knn = KNNBasic(k=grid_search_knn.best_params['rmse']['k'],
                    sim_options=grid_search_knn.best_params['rmse']['sim_options'])
best_knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [21]:
predictions = best_svd.test(testset)
svd_rmse =accuracy.rmse(predictions)
svd_rmse

RMSE: 0.8809


0.8809493546126119

In [22]:
knn_predictions = best_knn.test(testset)
knn_rmse = accuracy.rmse(knn_predictions)
knn_rmse

RMSE: 0.9474


0.9474474763274131

# Make recommendation

In [34]:
def get_recommendations(model, user_id, num_recommendations):
    unique_users = merged_data['userId'].unique()
    unique_movies = merged_data['movieId'].unique()

# Dictionary to store top N recommendations for each user
    top_n_recommendations = {}

# Predict ratings for each user-movie pair
    user_recommendations = []
    for movie in unique_movies:
        # Predict rating for the (user, movie) pair
        pred = knn_model.predict(user_id, movie, verbose=False)
        user_recommendations.append((movie, pred.est))
    
        # Sort movies by predicted rating and get top N (e.g., top 5)
        top_n = sorted(user_recommendations, key=lambda x: x[1], reverse=True)[:num_recommendations]
    
        # Store top N recommendations for the user
        top_n_recommendations[user] = top_n

    # Output top N recommendations for each user
    print(f"Top {num_recommendations} recommendations for User {user_id}:")
    for movie, predicted_rating in top_n:
        title = merged_data.loc[merged_data['movieId'] == movie, 'title']
        print(f"{title.values[0]}. Predicted Rating {predicted_rating:.2f}")

In [44]:
recommended_movies = get_recommendations(best_svd, user_id=98, num_recommendations=5)

Top 5 recommendations for User 98:
Nick of Time. Predicted Rating 5.00
The Crazy Stranger. Predicted Rating 5.00
The Inquisitor. Predicted Rating 5.00
Posti in piedi in paradiso. Predicted Rating 5.00
The End of Poverty?. Predicted Rating 5.00


In [46]:
recommended_movies = get_recommendations(best_knn, user_id=48, num_recommendations=5)

Top 5 recommendations for User 48:
Nick of Time. Predicted Rating 5.00
The Crazy Stranger. Predicted Rating 5.00
Ninotchka. Predicted Rating 5.00
The Inquisitor. Predicted Rating 5.00
The Garden of the Finzi-Continis. Predicted Rating 5.00
